In [12]:
# combining Costs file with Markets
# note that we will merge on the efficient measure name and the market at that is what 
# defines a measure (which is how we did the avoided costs (at the measure level)). However the market and 
# adoption model are at the condition level so after the merge we will have a lot of NAs 
# all the existing and baseline equipment will have NAs
# we will need to make sure they get pulled into the adoption model correctly
# the flags will need to force this
# this step is needed to get all the information the adoption model needs in one dataframe

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [14]:
df = pd.read_pickle("020_input/measure_costs_benefits.plk")
df

,measure_name,sector,program,electric_utility,gas_utility,market,baseline_condition,efficient_condition,building_type,measure_life_(yrs),...,TRC_BCR,SCT_cost,SCT_benefit,SCT_BCR,RIM_cost,RIM_benefit,RIM_BCR,PCT_cost,PCT_benefit,PCT_BCR
0,refrigerator_electricity_efficient_residential...,residential,LIRRet,test_utility_1,test_utility_2,RET_ER,refrigerator_electricity_existing_residential,refrigerator_electricity_efficient_residential,single_family_li,10,...,0.045755,2640.0,120.792465,0.045755,1980.0,960.0,0.484848,1200,2195.066425,1.829222
1,refrigerator_electricity_efficient_residential...,residential,NLIRRet,test_utility_1,test_utility_2,RET_ER,refrigerator_electricity_existing_residential,refrigerator_electricity_efficient_residential,multi_family,10,...,0.045755,2640.0,120.792465,0.045755,1980.0,960.0,0.484848,1200,2195.066425,1.829222
2,refrigerator_electricity_efficient_residential...,residential,LIRRet,test_utility_1,test_utility_2,RET_ER,refrigerator_electricity_existing_residential,refrigerator_electricity_efficient_residential,multi_family_li,10,...,0.045755,2640.0,120.792465,0.045755,1980.0,960.0,0.484848,1200,2195.066425,1.829222
3,refrigerator_electricity_efficient_residential...,residential,NLIRRet,test_utility_1,test_utility_2,RET_ER,refrigerator_electricity_existing_residential,refrigerator_electricity_efficient_residential,single_family,10,...,0.045755,2640.0,120.792465,0.045755,1980.0,960.0,0.484848,1200,2195.066425,1.829222
4,refrigerator_electricity_efficient_residential...,residential,LIRNC,test_utility_1,test_utility_2,NC,refrigerator_electricity_baseline_residential,refrigerator_electricity_efficient_residential,single_family_li,10,...,0.119082,880.0,104.792465,0.119082,660.0,320.0,0.484848,400,795.066425,1.987666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,refrigerator_electricity_top10_residential_ref...,residential,NLIRRepl,none,test_utility_3,ROB,refrigerator_electricity_baseline_residential,refrigerator_electricity_top10_residential,single_family,10,...,0.070896,1680.0,119.106047,0.070896,770.0,560.0,0.727273,700,803.380006,1.147686
124,refrigerator_electricity_top10_residential_ref...,residential,LIRRepl,none,test_utility_3,RENO,refrigerator_electricity_baseline_residential,refrigerator_electricity_top10_residential,single_family_li,10,...,0.070896,1680.0,119.106047,0.070896,770.0,560.0,0.727273,700,803.380006,1.147686
125,refrigerator_electricity_top10_residential_ref...,residential,NLIRRepl,none,test_utility_3,RENO,refrigerator_electricity_baseline_residential,refrigerator_electricity_top10_residential,multi_family,10,...,0.070896,1680.0,119.106047,0.070896,770.0,560.0,0.727273,700,803.380006,1.147686
126,refrigerator_electricity_top10_residential_ref...,residential,LIRRepl,none,test_utility_3,RENO,refrigerator_electricity_baseline_residential,refrigerator_electricity_top10_residential,multi_family_li,10,...,0.070896,1680.0,119.106047,0.070896,770.0,560.0,0.727273,700,803.380006,1.147686


In [15]:
### making a tool###
# # make a list from df which is efficient_condition and measure_life_(yrs) and drop duplicates

#this will have to come from the workpapers 
df_eff_life = df[['efficient_condition', 'measure_life_(yrs)']].drop_duplicates()
df_base_life = df[['baseline_condition', 'measure_life_(yrs)']].drop_duplicates()
# now just stack these two dataframes on top of each other with the columns efficient_condition and baseline_condition renamed to condition
df_eff_life = df_eff_life.rename(columns={'efficient_condition': 'condition', 'measure_life_(yrs)': 'measure_life_(yrs)'})
df_base_life = df_base_life.rename(columns={'baseline_condition': 'condition', 'measure_life_(yrs)': 'measure_life_(yrs)'})

df_life = pd.concat([df_eff_life, df_base_life]).drop_duplicates().reset_index(drop=True)
df_life

,condition,measure_life_(yrs)
0,refrigerator_electricity_efficient_residential,10
1,refrigerator_electricity_top10_residential,10
2,refrigerator_electricity_existing_residential,10
3,refrigerator_electricity_baseline_residential,10


In [16]:
df_yr1 = pd.read_pickle("020_input/df_yr1_001.pkl")
# Apply filters
# df_yr1 = df_yr1[
#     (df_yr1['electric_utility'] == 'test_utility_1') &
#     (df_yr1['gas_utility'] == 'test_utility_1') &
#     (df_yr1['competition_group'] == 'refrigeration')
# ]
# df_yr1

In [17]:
df_yr1 = pd.read_pickle("020_input/df_yr1_001.pkl")
# first need to change df_yr1 into long format with the current column names which are building types into rows and a new column for building type
df_yr1 = df_yr1.melt(id_vars=['condition_name', 'competition_group', 'subgroup', 'electric_utility', 'gas_utility'], var_name='building_type', value_name='initial_count')

df_yr1

,condition_name,competition_group,subgroup,electric_utility,gas_utility,building_type,initial_count
0,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_1,test_utility_1,single_family_year_one,10000.0
1,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_1,test_utility_2,single_family_year_one,20000.0
2,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_2,test_utility_2,single_family_year_one,55000.0
3,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_1,none,single_family_year_one,10000.0
4,furnace_fuel_oil_existing_residential,heating_cooling,oil_furnace,test_utility_2,none,single_family_year_one,11000.0
...,...,...,...,...,...,...,...
499,HERS_electric_baseline_residential,whole_home,none,test_utility_2,test_utility_2,multi_family_li_year_one,3500.0
500,HERS_electric_baseline_residential,whole_home,none,test_utility_1,none,multi_family_li_year_one,1250.0
501,HERS_electric_baseline_residential,whole_home,none,test_utility_2,none,multi_family_li_year_one,1250.0
502,HERS_electric_baseline_residential,whole_home,none,none,test_utility_1,multi_family_li_year_one,1250.0


In [18]:
# use df_life to map measure life to df_yr1 based on efficient_condition
df_yr1 = pd.merge(df_yr1, df_life, left_on='condition_name', right_on='condition', how='left')

# now I am going to allocate the initial counts into the markets based on the stock and flow proportions from expert opinion
# there are three markets that I want to allocate into: RET_ER, REMAINING, and ROB 
# ROB is 1/EUL (measure_life (yrs)
# RET_ER is 1/3 of non ROB stock
# remaining is 2/3 of non ROB stock
df_yr1['ROB'] = df_yr1['initial_count'] / df_yr1['measure_life_(yrs)']  # assuming measure life of 15 years
####################################
######### Feature add ##############
####################################
# We need to also include Ret_add_on
# the adoption model is ready for this
df_yr1['RET_ADD_ON'] = (df_yr1['initial_count'] - df_yr1['ROB']) 
# RET_ADD_ON is all of the non ROB stock



df_yr1['RET_ER'] = (df_yr1['initial_count'] - df_yr1['ROB']) * (1/3)
df_yr1['REMAINING'] = (df_yr1['initial_count'] - df_yr1['ROB']) * (2/3)

df_yr1 = df_yr1.melt(id_vars=['condition_name', 'competition_group', 'subgroup', 'electric_utility', 'gas_utility', 'building_type', 'measure_life_(yrs)', 'initial_count', 'condition'], var_name='market', value_name='count')
#need to remove the text '_year_one' from the building_type column
df_yr1['building_type'] = df_yr1['building_type'].str.replace('_year_one', '')

In [19]:
# we need to add a check to make sure the Market Characterization sheet and the work papers have the same conditions in them
# we also need to add a check to make sure all existing and baseline versions are included as needed
# for now remove all nas on measure_life_(yrs)
df_yr1 = df_yr1.dropna(subset=['measure_life_(yrs)'])
df_yr1

,condition_name,competition_group,subgroup,electric_utility,gas_utility,building_type,measure_life_(yrs),initial_count,condition,market,count
56,refrigerator_electricity_existing_residential,refrigeration,full_size,test_utility_1,test_utility_1,single_family,10.0,10000.0,refrigerator_electricity_existing_residential,ROB,1000.0
57,refrigerator_electricity_existing_residential,refrigeration,full_size,test_utility_1,test_utility_2,single_family,10.0,20000.0,refrigerator_electricity_existing_residential,ROB,2000.0
58,refrigerator_electricity_existing_residential,refrigeration,full_size,test_utility_2,test_utility_2,single_family,10.0,55000.0,refrigerator_electricity_existing_residential,ROB,5500.0
59,refrigerator_electricity_existing_residential,refrigeration,full_size,test_utility_1,none,single_family,10.0,10000.0,refrigerator_electricity_existing_residential,ROB,1000.0
60,refrigerator_electricity_existing_residential,refrigeration,full_size,test_utility_2,none,single_family,10.0,11000.0,refrigerator_electricity_existing_residential,ROB,1100.0
...,...,...,...,...,...,...,...,...,...,...,...
1969,refrigerator_electricity_top10_residential,refrigeration,full_size,test_utility_2,test_utility_2,multi_family_li,10.0,2100.0,refrigerator_electricity_top10_residential,REMAINING,1260.0
1970,refrigerator_electricity_top10_residential,refrigeration,full_size,test_utility_1,none,multi_family_li,10.0,750.0,refrigerator_electricity_top10_residential,REMAINING,450.0
1971,refrigerator_electricity_top10_residential,refrigeration,full_size,test_utility_2,none,multi_family_li,10.0,750.0,refrigerator_electricity_top10_residential,REMAINING,450.0
1972,refrigerator_electricity_top10_residential,refrigeration,full_size,none,test_utility_1,multi_family_li,10.0,750.0,refrigerator_electricity_top10_residential,REMAINING,450.0


In [20]:
# now we just merge df_yr1 and df on condition_name, competition_group, electric_utility, gas_utility, building_type, and market
df_yrs = pd.merge(df_yr1, df, left_on=['condition_name', 'electric_utility', 'gas_utility', 'building_type', 'market', 'measure_life_(yrs)'], right_on=['efficient_condition',  'electric_utility', 'gas_utility', 'building_type', 'market', 'measure_life_(yrs)'], how='left')
df_yrs 


,condition_name,competition_group,subgroup,electric_utility,gas_utility,building_type,measure_life_(yrs),initial_count,condition,market,...,TRC_BCR,SCT_cost,SCT_benefit,SCT_BCR,RIM_cost,RIM_benefit,RIM_BCR,PCT_cost,PCT_benefit,PCT_BCR
0,refrigerator_electricity_existing_residential,refrigeration,full_size,test_utility_1,test_utility_1,single_family,10.0,10000.0,refrigerator_electricity_existing_residential,ROB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,refrigerator_electricity_existing_residential,refrigeration,full_size,test_utility_1,test_utility_2,single_family,10.0,20000.0,refrigerator_electricity_existing_residential,ROB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,refrigerator_electricity_existing_residential,refrigeration,full_size,test_utility_2,test_utility_2,single_family,10.0,55000.0,refrigerator_electricity_existing_residential,ROB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,refrigerator_electricity_existing_residential,refrigeration,full_size,test_utility_1,none,single_family,10.0,10000.0,refrigerator_electricity_existing_residential,ROB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,refrigerator_electricity_existing_residential,refrigeration,full_size,test_utility_2,none,single_family,10.0,11000.0,refrigerator_electricity_existing_residential,ROB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,refrigerator_electricity_top10_residential,refrigeration,full_size,test_utility_2,test_utility_2,multi_family_li,10.0,2100.0,refrigerator_electricity_top10_residential,REMAINING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
444,refrigerator_electricity_top10_residential,refrigeration,full_size,test_utility_1,none,multi_family_li,10.0,750.0,refrigerator_electricity_top10_residential,REMAINING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445,refrigerator_electricity_top10_residential,refrigeration,full_size,test_utility_2,none,multi_family_li,10.0,750.0,refrigerator_electricity_top10_residential,REMAINING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
446,refrigerator_electricity_top10_residential,refrigeration,full_size,none,test_utility_1,multi_family_li,10.0,750.0,refrigerator_electricity_top10_residential,REMAINING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#Not removing NAs we need to make sure all the files have the shared column names and such

df_yrs.to_pickle("./020_output/df_yrs.pkl")
df_yrs.to_pickle("./030_input/df_yrs.pkl")